In [2]:
import pandas as pd
import numpy as np
df = pd.read_csv('/home/pampolha/Observações/S4G_1.csv')
mask_ra = (df['ra'] > 100) & (df['ra'] < 330)
mask_dec = (df['dec'] > (-60)) & (df['dec'] < 22)
mask_mstar = (df['mstar'] > 10) #MW analogs criteria
mask_vmaxg = (df['vmaxg'] > 100) #MW analogs criteria
mask_t = (df['t'] > 1.5) & (df['t'] < 4.5) #MW analogs criteria
mask_incl = (df['incl'] > 40) & (df['incl'] < 70) #MW analogs criteria

# Combine masks using the logic operator "&" (AND)
mask_combined = mask_ra & mask_dec & mask_mstar & mask_vmaxg & mask_t & mask_incl

# Remove lines that doesn't satisfy all criteria at the same time
df = df[mask_combined]

# List of galaxies to be removed (previous missions objects + actual)

galaxias_remover =  ( 'ESO404-012','NGC5750','NGC5757','NGC4045','IC4237', 'NGC5957','NGC5962','NGC3321', 
                    'IC1055','IC1067','IC1158','IC1438','NGC4897','NGC4902','NGC5913','NGC7219','NGC7690',
                    'NGC4480','NGC4378','NGC4409','NGC4580','PGC036217','NGC7421','NGC6870','NGC6902',
                    'NGC6923','NGC4941','NGC4050','NGC7418','NGC6118','IC4901','NGC6887','NGC6063',
                    'NGC6925','NGC6070','NGC2731','NGC3507','NGC7205','NGC4158','NGC2966','NGC4527','NGC4351',
                    'NGC5247','NGC5756','NGC7552','NGC7179','NGC3976','NGC4628','NGC4067','NGC5364','NGC4313',
                    'UGC08067','NGC3715','NGC3177','ESO443-042','NGC7368','NGC4639','IC5334','NGC4343','PGC047721',
                    'NGC4394','NGC3681','NGC4462','NGC4746','NGC7171','NGC5534','NGC3705','NGC3684', 'NGC5806',
                    'NGC5850', 'NGC5878','NGC5915','NGC5921','NGC5937','NGC7163','NGC6012','NGC7140','ESO342-013', ) 

# Filters the DataFrame keeping only the galaxies that are NOT in the removal list
df_filtrado = df[~df['object'].isin(galaxias_remover)]

# Show how many galaxies were removed
print(f"Total original: {len(df)}")
print(f"Total após filtro: {len(df_filtrado)}")
print(f"Galáxias removidas: {len(df) - len(df_filtrado)}")

# Save the result (if necessary)
df_filtrado.to_csv('tabela_filtrada.csv', index=False)
df_filtrado[:]

Total original: 46
Total após filtro: 34
Galáxias removidas: 12


,object,ra,dec,sma1_25p5,pa1_25p5,ellip1_25p5,sma1_26p5,pa1_26p5,ellip1_26p5,sma2_25p5,...,v3k,modz,mod0,mabs,summary,x,y,z,spt_ind,cntr
15,NGC4658,191.157310,-10.083328,66.8,2.4,0.412,75.9,23.5,0.202,61.7,...,2733.3,32.698,31.660,-20.284,"<a href=""/data/SPITZER/S4G/galaxies/NGC4658.ht...",-0.965946,-0.190515,-0.175080,120030032,1267
166,NGC2775,137.583953,7.037979,218.6,-26.9,0.076,261.9,-37.9,0.216,202.9,...,1651.9,31.517,33.183,-20.695,"<a href=""/data/SPITZER/S4G/galaxies/NGC2775.ht...",-0.732704,0.669427,0.122527,223103233,734
184,NGC2903,143.042403,21.501831,365.5,19.4,0.508,411.2,20.9,0.491,363.3,...,840.5,29.942,29.956,-21.115,"<a href=""/data/SPITZER/S4G/galaxies/NGC2903.ht...",-0.743469,0.559382,0.366531,223113313,752
215,NGC4699,192.259168,-8.664333,281.2,49.1,0.134,328.9,-8.2,0.046,245.9,...,1727.5,31.579,31.870,-21.534,"<a href=""/data/SPITZER/S4G/galaxies/NGC4699.ht...",-0.966045,-0.209911,-0.150645,120022023,1279
299,NGC4981,197.203149,-6.777515,113.0,-28.2,0.411,121.3,-34.4,0.333,102.6,...,1997.0,31.968,31.706,-20.314,"<a href=""/data/SPITZER/S4G/galaxies/NGC4981.ht...",-0.948587,-0.293694,-0.118014,120311202,1339
301,NGC4995,197.419343,-7.833410,133.1,84.6,0.334,NaN,NaN,NaN,116.0,...,2061.0,32.067,32.400,-20.663,"<a href=""/data/SPITZER/S4G/galaxies/NGC4995.ht...",-0.945236,-0.296569,-0.136293,120311221,1341
312,NGC5054,199.243655,-16.634561,202.2,-21.8,0.387,221.6,-18.3,0.353,194.6,...,2053.1,32.018,30.924,-21.141,"<a href=""/data/SPITZER/S4G/galaxies/NGC5054.ht...",-0.904614,-0.315793,-0.286266,120332202,1352
317,NGC5079,199.908528,-12.698959,62.5,26.4,0.408,73.0,27.3,0.419,58.4,...,2610.7,32.623,NaN,-20.109,"<a href=""/data/SPITZER/S4G/galaxies/NGC5079.ht...",-0.917238,-0.332190,-0.219828,120332122,1357
388,NGC4165,183.049182,13.246485,45.7,-12.6,0.341,50.7,-15.7,0.345,54.6,...,2201.6,32.298,33.730,-18.398,"<a href=""/data/SPITZER/S4G/galaxies/NGC4165.ht...",-0.972015,-0.051778,0.229141,212021111,1056
396,NGC4193,183.473247,13.172955,87.2,-86.6,0.531,107.5,-86.7,0.553,79.9,...,2796.6,32.840,32.876,-20.354,"<a href=""/data/SPITZER/S4G/galaxies/NGC4193.ht...",-0.971898,-0.058988,0.227891,212021113,1064


In [3]:
# Display the first three columns of the filtered DataFrame (to use on Finders.ipynb notebook)
 print(df_filtrado.iloc[:, :3].to_string(index=False))

    object         ra        dec
   NGC4658 191.157310 -10.083328
   NGC2775 137.583953   7.037979
   NGC2903 143.042403  21.501831
   NGC4699 192.259168  -8.664333
   NGC4981 197.203149  -6.777515
   NGC4995 197.419343  -7.833410
   NGC5054 199.243655 -16.634561
   NGC5079 199.908528 -12.698959
   NGC4165 183.049182  13.246485
   NGC4193 183.473247  13.172955
   NGC3981 179.031044 -19.895948
   NGC4030 180.098393  -1.100035
   NGC5740 221.101844   1.679810
ESO576-001 197.598995 -21.684189
 PGC053093 223.043393 -10.740294
   NGC5781 224.172071 -17.243358
   NGC3368 161.690350  11.819975
   NGC3521 166.452415  -0.035937
   NGC4799 193.814776   2.896736
   NGC4826 194.181848  21.683075
   NGC3351 160.990618  11.703659
   NGC4450 187.123344  17.084958
   NGC5248 204.383361   8.885187
   NGC3627 170.062528  12.991503
   NGC4380 186.342423  10.016819
   NGC3673 171.303583 -26.736739
   NGC4237 184.297628  15.324096
   NGC5661 217.989218   6.250440
   NGC4568 189.142956  11.238855
   NGC4579